In [1]:
# Set up colab instance
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [2]:
!pip3 install pydicom
!git clone https://github.com/thomasp05/GIF-705-Projet.git

     |████████████████████████████████| 1.9MB 8.0MB/s 
Cloning into 'GIF-705-Projet'...
remote: Enumerating objects: 84, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 335 (delta 45), reused 51 (delta 24), pack-reused 251
Receiving objects: 100% (335/335), 3.76 MiB | 15.33 MiB/s, done.
Resolving deltas: 100% (186/186), done.


In [3]:
import os
os.chdir('drive/MyDrive/GIF-7005-Projet/gif-7005-projet/')

!git checkout features/DenseNet121
!git pull



M	train_collab.ipynb
Already on 'features/DenseNet121'
Your branch is up to date with 'origin/features/DenseNet121'.
Already up to date.


In [4]:
import time
import torch
from dataset import *
from models import *
from metrics import *
from grad_analysis import *
torch.manual_seed(111)

In [6]:
# HYPERPARAMETERS
N_EPOCH = 5
BATCH_SIZE = 4

In [ ]:
dataset = dcm_dataset('data')

train_set, test_set = train_test_split(dataset)
test_set.transforms = Downsample()

train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=4, num_workers=2)

test_loader = torch.utils.data.DataLoader(
    test_set, batch_size=1, num_workers=2)

In [ ]:
# check if gpu available 
print("CUDA available: ", torch.cuda.is_available()) 
device = torch.device("cuda" if(torch.cuda.is_available()) else "cpu") 

denseNet = DenseNet121(False).to(device)

# optim = torch.optim.Adam(denseNet.parameters(), lr=0.001)
optim = torch.optim.Adam(denseNet.parameters(), lr=1e-4, weight_decay=1e-3)
criterion = torch.nn.BCEWithLogitsLoss()

# If model exists, load it
# if os.path.exists("../drive/MyDrive/GIF-7005-Projet/gif-7005-projet/resnet18.pt"):
# denseNet.load_state_dict(torch.load("denseNet121_full_58_epochs.pt"))
# optim.load_state_dict(torch.load("denseNet121_optim_full_48_epochs.pt"))



CUDA available:  True


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0


In [ ]:
denseNet.train()

for epoch in range(N_EPOCH):
    timer = time.time()
    running_loss = 0 
    img_processed = 0

    for img, (target, bounding_box) in train_loader:
         
        optim.zero_grad()
    
        img, target = img.to(device), target.to(device)
      
        prediction = denseNet(img) 

        loss = criterion(prediction.squeeze(), target)

        loss.backward()

        optim.step()

        img_processed += img.shape[0]

        running_loss += loss.item() 

        # print(loss.item())
    print("Epoch : {}".format(epoch + 1))
    print("Epoch {} - training loss: {}".format(epoch,running_loss/len(train_loader)))
    print("Time elapsed : {}".format(time.time() - timer))
    torch.save(denseNet.state_dict(), "denseNet121_v2_5.pt")
    torch.save(optim.state_dict(), "denseNet121_optim_V2_5.pt")

Epoch : 1
Epoch 0 - training loss: 0.4192713398262314
Time elapsed : 3172.466762781143
Epoch : 2
Epoch 1 - training loss: 0.40184331794543526
Time elapsed : 855.9970769882202
Epoch : 3
Epoch 2 - training loss: 0.38913851180311915
Time elapsed : 852.8571398258209
Epoch : 4
Epoch 3 - training loss: 0.3748357436129869
Time elapsed : 857.5135958194733


KeyboardInterrupt: ignored

In [ ]:
# Script pour tester le densenet 
# check if gpu available 
print("CUDA available: ", torch.cuda.is_available()) 
device = torch.device("cuda" if(torch.cuda.is_available()) else "cpu") 

# loader le modele
denseNet = DenseNet121(True).to(device)

# Loader le state
denseNet.load_state_dict(torch.load("denseNet121_transfer_learning.pt"))

# eval mode 
denseNet.eval() 


CUDA available:  True


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0


DenseNet121(
  (upsample): Upsample(size=(224, 224), mode=bilinear)
  (base_model): DenseNet(
    (features): Sequential(
      (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu0): ReLU(inplace=True)
      (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (denseblock1): _DenseBlock(
        (denselayer1): _DenseLayer(
          (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace=True)
          (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu2): ReLU(inplace=True)
          (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
        (denselayer2): _DenseLay

In [ ]:
denseNet.eval()
run_test(denseNet, test_loader, [Confusion_matrix()], device)

tensor([[3940.,  672.],
        [ 219.,  505.]])


In [ ]:
# Script pour méthode par ensembles 
# check if gpu available 
print("CUDA available: ", torch.cuda.is_available()) 
device = torch.device("cuda" if(torch.cuda.is_available()) else "cpu") 

# loader les modeles
denseNet = DenseNet121(False).to(device)
resnet18 = Resnet_cam().to(device)
inceptionV3 = Inception_v3().to(device) 


# Load model states
denseNet.load_state_dict(torch.load("denseNet121_full_58_epochs.pt"))
resnet18.load_state_dict(torch.load("resnet18.pt"))
inceptionV3.load_state_dict(torch.load("inception_v3.pt"))

# eval mode 
denseNet.eval() 
resnet18.eval() 
inceptionV3.eval() 


CUDA available:  True


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0


Inception_v3(
  (upsample): Upsample(size=(299, 299), mode=bilinear)
  (base_model): Inception3(
    (Conv2d_1a_3x3): BasicConv2d(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_2a_3x3): BasicConv2d(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_2b_3x3): BasicConv2d(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (Conv2d_3b_1x1): BasicConv2d(
      (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=Tru

In [ ]:
# tester performance de l'ensemble 
# run_test_ensemble3(denseNet, resnet18, inceptionV3, test_loader, [Confusion_matrix()], device)
# run_test_ensemble2(denseNet, inceptionV3, test_loader, [Confusion_matrix()], device)

run_test(inceptionV3, test_loader, [Confusion_matrix()], device)

tensor([[4015.,  706.],
        [ 144.,  471.]])
